In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# Examine the data
print(dataset.shape)
print(dataset.describe())
print(dataset.dtypes)

In [ ]:
# Check for missing values
print(dataset.isna().sum())

In [ ]:
# Convert categorical data to numbers
# For 'sex' column
dataset['sex'] = dataset['sex'].map({'female': 0, 'male': 1})


In [ ]:
# For 'smoker' column
dataset['smoker'] = dataset['smoker'].map({'no': 0, 'yes': 1})

In [ ]:
# For 'region' column - using one-hot encoding
region_one_hot = pd.get_dummies(dataset['region'], prefix='region')
dataset = pd.concat([dataset, region_one_hot], axis=1)
dataset.drop('region', axis=1, inplace=True)

print(dataset.head())

In [ ]:
# Split the data into train and test sets
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)


In [ ]:
# Pop off the expenses column to create labels
train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')

print(f"Training set size: {train_dataset.shape}")
print(f"Test set size: {test_dataset.shape}")


In [ ]:
# Normalize the input features
normalizer = preprocessing.Normalization()
normalizer.adapt(np.array(train_dataset))

In [ ]:
# Build the model
def build_and_compile_model():
    model = keras.Sequential([
        normalizer,
        layers.Dense(64, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)
    ])

    model.compile(loss='mean_absolute_error',
                 optimizer=tf.keras.optimizers.Adam(0.001),
                 metrics=['mean_absolute_error'])
    return model

In [ ]:
# Create and compile the model
model = build_and_compile_model()
model.summary()

In [ ]:
# Train the model
history = model.fit(
    train_dataset,
    train_labels,
    validation_split=0.2,
    verbose=1,
    epochs=100,
    callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)]
)

In [ ]:
# Visualize the training progress
def plot_loss(history):
    plt.figure(figsize=(10, 6))
    plt.plot(history.history['mean_absolute_error'], label='MAE')
    plt.plot(history.history['val_mean_absolute_error'], label='Val MAE')
    plt.xlabel('Epoch')
    plt.ylabel('Mean Absolute Error ($)')
    plt.title('Model Training Performance')
    plt.legend()
    plt.grid(True)
    plt.show()

plot_loss(history)

In [ ]:
# Evaluate the model on the test data
test_results = model.evaluate(test_dataset, test_labels, verbose=0)
print(f"Test MAE: ${test_results[1]:.2f}")

In [ ]:
# Predict on test data
test_predictions = model.predict(test_dataset).flatten()

In [ ]:
# Plot predictions vs actual
plt.figure(figsize=(10, 8))
plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values ($)')
plt.ylabel('Predictions ($)')
plt.title('Predictions vs Actual')
plt.axis('equal')
plt.axis('square')
plt.xlim([0, plt.xlim()[1]])
plt.ylim([0, plt.ylim()[1]])
plt.plot([-100, plt.xlim()[1]], [-100, plt.ylim()[1]])
plt.show()

In [ ]:
# Plot prediction error
error = test_predictions - test_labels
plt.figure(figsize=(10, 6))
plt.hist(error, bins=25)
plt.xlabel('Prediction Error ($)')
plt.ylabel('Count')
plt.title('Prediction Error Distribution')
plt.show()

In [ ]:
# Check if our model meets the requirement (MAE < 3500)
if test_results[1] < 3500:
    print(f"Success! MAE is ${test_results[1]:.2f}, which is under the $3500 threshold.")
else:
    print(f"Need improvement. MAE is ${test_results[1]:.2f}, which is above the $3500 threshold.")

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)


In [ ]:
# Test against the test dataset
error = model.evaluate(test_dataset, test_labels, verbose=0)
print(f"MAE: ${error[1]:.2f}")

In [ ]:
# Make predictions with the test dataset
test_predictions = model.predict(test_dataset).flatten()

In [ ]:
# Plot predictions against actual values
a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)